In [1]:
import pandas as pd
import numpy as np

In [2]:
census_2021 = pd.read_csv('housingandsanitation_10%_20221011d.csv')

/tmp/ipykernel_119/413822960.py:1: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  census_2021 = pd.read_csv('housingandsanitation_10%_20221011d.csv')


In [3]:
census_2021.columns

Index(['nqid', 'region', 'distcode', 'subdist', 'urbrur', 'restype', 'a11',
       'a12', 'e01', 'mr01', 'usual_males', 'usual_females', 'usual_members',
       'visiting_males', 'visiting_females', 'visitors', 'absent_males',
       'absent_females', 'absent_members', 'enummales', 'enumfemales',
       'totpopn', 'emigrant_males', 'emigrant_females', 'emigrants',
       'deceased_males', 'deceased_females', 'deaths', 'h01', 'h02', 'h03',
       'h04', 'h05', 'h06', 'h07a', 'h07b', 'h08a', 'h08b', 'h09a', 'h09b',
       'h10', 'h11a', 'h11b', 'h11c', 'h12', 'h13a1', 'h13a2', 'h13b1',
       'h13b2', 'h13c1', 'h13c2', 'h13d1', 'h13d2', 'h13e1', 'h13e2', 'h13f1',
       'h13f2', 'h13g1', 'h13g2', 'h13h1', 'h13h2', 'h13i1', 'h13i2', 'h13j1',
       'h13j2', 'h13k1', 'h13k2', 'h13l1', 'h13l2', 'h13m1', 'h13m2', 'h13n1',
       'h13n2', 'h13o1', 'h13o2', 'h13p1', 'h13p2', 'h13q1', 'h13q2', 'h13r1',
       'h13r2', 'h13s1', 'h13s2', 'newh03', 's01', 's02', 's03', 's04', 's05',
       's06', 

In [4]:
column_names = {'h01': 'dwelling_type', 'h02': 'wall_material', 'h03': 'roof_material', 'h04': 'floor_material', 'h05': 'tenancy', 'h06': 'ownership', 'h07a': 'rooms', 'newh03': 'new_roof_material'}
census_2021 = census_2021.rename(columns=column_names)

def g(df, selected_columns):
    df.set_index(['region','distcode', 'subdist'], inplace=True)
    grouped = df.groupby(level=[0, 1, 2])
    return {col: grouped[col].value_counts().unstack(fill_value=0) for col in selected_columns}

selected_columns = list(column_names.values())  # Example; Modify with your desired columns
# print(selected_columns)

result = g(census_2021.copy(), selected_columns.copy())

for col, df_ in result.items():
    # print(f"Value counts for column {col}")
    df_.to_csv(col+'.csv')

In [5]:
column_names = {'h01': 'dwelling_type', 'h02': 'wall_material', 'h03': 'roof_material', 
                'h04': 'floor_material', 'h05': 'tenancy', 'h06': 'ownership', 
                'h07a': 'rooms', 'newh03': 'new_roof_material'}
census_2021 = census_2021.rename(columns=column_names)

def g(df, selected_columns):
    df.set_index(['region', 'distcode', 'subdist'], inplace=True)
    grouped = df.groupby(level=[0, 1, 2])
    result = {}
    for col in sorted(selected_columns):  # Ensure alphabetical order
        # Count including NaN values, then fill NaNs in the result with 0
        counts = grouped[col].value_counts(dropna=False).unstack(fill_value=0)
        
        # Separately calculate the NaN count
        na_count = counts.pop(np.nan) if np.nan in counts else 0
        
        # Assign the NaN count to the 'NA' column
        counts['NA'] = na_count
        
        result[col] = counts
    return result

selected_columns = list(column_names.values())  # Example; Modify with your desired columns

result = g(census_2021.copy(), selected_columns.copy())

for col, df_ in result.items():
    df_.to_csv('Python_outputs/'+col+'.csv')



In [3]:
column_names = {'h01': 'dwelling_type', 'h02': 'wall_material', 'h03': 'roof_material', 
                'h04': 'floor_material', 'h05': 'tenancy', 'h06': 'ownership', 
                'h07a': 'rooms', 'newh03': 'new_roof_material'}
census_2021 = census_2021.rename(columns=column_names)

def g(df, selected_columns):
    df.set_index(['region', 'distcode', 'subdist'], inplace=True)
    grouped = df.groupby(level=[0, 1, 2])
    result = {}
    for col in selected_columns:  # Sorting happens after adding 'NA', ensuring all names are strings
        # Count including NaN values, then fill NaNs in the result with 0
        counts = grouped[col].value_counts(dropna=False).unstack(fill_value=0)
        
        # Separately calculate the NaN count and insert it into the DataFrame
        if np.nan in counts:
            counts['NA'] = counts.pop(np.nan)
        else:
            counts['NA'] = 0

        # Convert all column names to strings to avoid TypeError during sorting
        counts.columns = counts.columns.astype(str)
        
        # Now sort the columns alphabetically, including 'NA'
        counts = counts.reindex(sorted(counts.columns), axis=1)
        
        result[col] = counts
    return result

selected_columns = list(column_names.values())  # Use the renamed column names

result = g(census_2021.copy(), selected_columns.copy())

for col, df_ in result.items():
    df_.to_csv('Python_outputs/'+col+'.csv')